In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [4]:
df_train = pd.read_csv("training_data_category.csv")
df_train

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category
0,0,0,32,221.0,6.0,40
1,0,0,33,347.0,3.0,37
2,0,0,35,247.0,1.0,40
3,0,0,43,221.0,1.0,40
4,0,0,51,127.0,1.0,57
...,...,...,...,...,...,...
1739017,33,59,22087,119.0,6.0,83
1739018,33,59,22088,119.0,2.0,83
1739019,33,59,22091,179.0,1.0,83
1739020,33,59,22100,629.0,1.0,42


In [16]:
item_mean = df_train.groupby(['date_block_num','item_id'],as_index=True, sort=False, group_keys=True)['item_cnt_day'].mean()
item_mean[(0, 32)]

7.475

In [32]:
shop_mean = df_train.groupby(['date_block_num','shop_id'],as_index=True, sort=False, group_keys=True)['item_cnt_day'].mean()
# shop_mean.set_index('date_block_num','shop_id')
# shop_mean.get_group((0,4))
shop_mean

date_block_num  shop_id
0               0          2.310688
                1          1.898840
                2          1.507895
                3          1.392015
                4          1.851138
                             ...   
33              55         4.065979
                56         1.551597
                57         1.845418
                58         1.742169
                59         1.501901
Name: item_cnt_day, Length: 1586, dtype: float64

In [33]:
# Need to add the item_cnt_mean for each item per date block
def add_item_mean(row):
    it_id = int(row['item_id'])
    dt_block = int(row['date_block_num'])
    mean_val = item_mean[(dt_block,it_id)]
    return mean_val

In [34]:
# Need to add the shop_cnt_mean for each shop per date block
def add_shop_mean(row):
    shp_id = int(row['shop_id'])
    dt_block = int(row['date_block_num'])
    mean_val = shop_mean[(dt_block,shp_id)]
    return mean_val

In [21]:
df_train['item_mean'] = df_train.apply(add_item_mean, axis = 1)

In [27]:
df_train

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category,item_mean
0,0,0,32,221.0,6.0,40,7.475000
1,0,0,33,347.0,3.0,37,2.033333
2,0,0,35,247.0,1.0,40,3.250000
3,0,0,43,221.0,1.0,40,1.000000
4,0,0,51,127.0,1.0,57,1.444444
...,...,...,...,...,...,...,...
1739017,33,59,22087,119.0,6.0,83,3.346154
1739018,33,59,22088,119.0,2.0,83,5.285714
1739019,33,59,22091,179.0,1.0,83,1.500000
1739020,33,59,22100,629.0,1.0,42,4.787879


In [35]:
df_train['shop_mean'] = df_train.apply(add_shop_mean, axis = 1)

In [36]:
df_train.to_csv('training_data_cat_mean.csv', index=False)

In [40]:
df = df_train[['shop_id','item_id','date_block_num','item_category','item_price','item_mean','shop_mean','item_cnt_day']]
d = df.rename(columns={"item_cnt_day": "item_cnt_month"})
d

,shop_id,item_id,date_block_num,item_category,item_price,item_mean,shop_mean,item_cnt_month
0,0,32,0,40,221.0,7.475000,2.310688,6.0
1,0,33,0,37,347.0,2.033333,2.310688,3.0
2,0,35,0,40,247.0,3.250000,2.310688,1.0
3,0,43,0,40,221.0,1.000000,2.310688,1.0
4,0,51,0,57,127.0,1.444444,2.310688,1.0
...,...,...,...,...,...,...,...,...
1739017,59,22087,33,83,119.0,3.346154,1.501901,6.0
1739018,59,22088,33,83,119.0,5.285714,1.501901,2.0
1739019,59,22091,33,83,179.0,1.500000,1.501901,1.0
1739020,59,22100,33,42,629.0,4.787879,1.501901,1.0


In [41]:
d.to_csv('training_data_cat_mean.csv', index=False)

In [42]:
# No. of days you want to predict the sales
forecast_out = 1
df=d
df['shift_item_cnt'] = d[['item_cnt_month']].shift(-forecast_out)
df

,shop_id,item_id,date_block_num,item_category,item_price,item_mean,shop_mean,item_cnt_month,shift_item_cnt
0,0,32,0,40,221.0,7.475000,2.310688,6.0,3.0
1,0,33,0,37,347.0,2.033333,2.310688,3.0,1.0
2,0,35,0,40,247.0,3.250000,2.310688,1.0,1.0
3,0,43,0,40,221.0,1.000000,2.310688,1.0,1.0
4,0,51,0,57,127.0,1.444444,2.310688,1.0,1.0
...,...,...,...,...,...,...,...,...,...
1739017,59,22087,33,83,119.0,3.346154,1.501901,6.0,2.0
1739018,59,22088,33,83,119.0,5.285714,1.501901,2.0,1.0
1739019,59,22091,33,83,179.0,1.500000,1.501901,1.0,1.0
1739020,59,22100,33,42,629.0,4.787879,1.501901,1.0,1.0


In [51]:
x = np.array(df.drop(['shift_item_cnt'],1))
x = x[:-forecast_out]
# x = x[:int(len(x)/100)]
len(x)

1739021

In [52]:
y = np.array(df['shift_item_cnt'])
y = y[:-forecast_out]
# y = y[:int(len(y)/100)]
len(y)

1739021

In [53]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [54]:
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)
lr_confidence = lr.score(x_test,y_test)
lr_confidence

0.0660977791320464

In [55]:
# set x_forecast euqal to last n days of original dataset
# x_forecast = df.drop(['shift_item_cnt'],1)[-forecast_out:]
lr_prediction = lr.predict(x_test)
lr_prediction
RMSE = np.sqrt(metrics.mean_squared_error(y_test, lr_prediction))
print("RMSE:", RMSE)

RMSE: 7.069701381151726


In [ ]:
svr = SVR(kernel='rbf', C=1e3, gamma=0.1, cache_size=7000)
svr.fit(x_train,y_train)

In [49]:
svm_confidence = svr.score(x_test, y_test)
svm_confidence

-0.1438009173146948

In [50]:
svm_prediction = svr.predict(x_test)
svm_prediction
RMSE = np.sqrt(metrics.mean_squared_error(y_test, svm_prediction))
print("RMSE:", RMSE)

RMSE: 2.2989662149072116
